# Mesh generation for tutorial 06, cases 1a, 2a, 3a, 4a-b, 5, 6, 8

This file generates the mesh which is used in the following examples:
* 1a_poisson
* 2a_advection_diffusion_reaction
* 3a_advection_diffusion_reaction_neumann_control
* 4a_poisson_dirichlet_control
* 4b_poisson_neumann_control_boundary_observation
* 5_stokes
* 6_navier_stokes
* 8_navier_stokes_neumann_control

In [ ]:
import dolfinx.io
import dolfinx.mesh
import mpi4py
import numpy as np

In [ ]:
import multiphenicsx.io
import multiphenicsx.mesh

### Create mesh

In [ ]:
mesh = dolfinx.mesh.create_unit_square(mpi4py.MPI.COMM_WORLD, 32, 32)

In [ ]:
multiphenicsx.io.plot_mesh(mesh)

### Create (trivial) subdomains

In [ ]:
num_cells = mesh.topology.index_map(mesh.topology.dim).size_local
subdomains = dolfinx.mesh.MeshTags(
    mesh, mesh.topology.dim, np.arange(0, num_cells), np.zeros((num_cells,), dtype=np.intc))
subdomains.name = "subdomains"

### Create boundaries

In [ ]:
def bottom(x: np.typing.NDArray[np.float64]) -> np.typing.NDArray[bool]:
    """Condition that defines the bottom boundary."""
    return abs(x[1] - 0.) < np.finfo(float).eps


def left(x: np.typing.NDArray[np.float64]) -> np.typing.NDArray[bool]:
    """Condition that defines the left boundary."""
    return abs(x[0] - 0.) < np.finfo(float).eps


def top(x: np.typing.NDArray[np.float64]) -> np.typing.NDArray[bool]:
    """Condition that defines the top boundary."""
    return abs(x[1] - 1.) < np.finfo(float).eps


def right(x: np.typing.NDArray[np.float64]) -> np.typing.NDArray[bool]:
    """Condition that defines the right boundary."""
    return abs(x[0] - 1.) < np.finfo(float).eps


bottom.id = 1
left.id = 2
top.id = 3
right.id = 4

boundaries_entities = dict()
boundaries_values = dict()
for boundary in (bottom, left, top, right):
    boundaries_entities[boundary.id] = dolfinx.mesh.locate_entities_boundary(
        mesh, mesh.topology.dim - 1, boundary)
    boundaries_values[boundary.id] = np.full(
        boundaries_entities[boundary.id].shape, boundary.id, dtype=np.intc)
boundaries = dolfinx.mesh.MeshTags(
    mesh, mesh.topology.dim - 1,
    np.hstack(list(boundaries_entities.values())), np.hstack(list(boundaries_values.values())))
boundaries.name = "boundaries"

In [ ]:
multiphenicsx.io.plot_mesh_tags(boundaries)

### Save

In [ ]:
with dolfinx.io.XDMFFile(mpi4py.MPI.COMM_WORLD, "square.xdmf", "w") as output:
    output.write_mesh(mesh)
    output.write_meshtags(subdomains)
    output.write_meshtags(boundaries)